# Analytical linear fit:
This Python macro illustrates how to perform a linear fit **analytically**, i.e. without any minimisation (e.g. by Minuit), and thus much faster. This can be done, since one can differentiate the Chi2, set this to zero and solve for both intercept and slope.

## References:
- Note on course webpage
- Bevington: Chapter 6

## Author(s), contact(s), and dates:
- Author: Troels C. Petersen (NBI)
- Email:  petersen@nbi.dk
- Date:   14th of November 2024

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
from iminuit import Minuit
from iminuit.cost import LeastSquares

### Program settings:

In [ ]:
r = np.random                         # Random generator
r.seed(42)                            # Set a random seed (but a fixed one - more on that later.)
save_plots = False

In [ ]:
Nexp = 10       # Number of datasets fitted
Npoints = 9
alpha0 = 3.6
alpha1 = 0.3
sigmay = 1.0

# Generating and fitting data:
In the following, we generate data points following a simple line and fit these both **analytically** and **numerically**. The linear fit (and only this) can be done analytically, as discussed in Barlow chapter 6.2 and outlined here: http://www.nbi.dk/~petersen/Teaching/Stat2024/Notes/StraightLineFit.pdf.
The numerical fit of the line (and any other function) is done iteratively by Minuit. The code is slightly shorter, but a lot slower, as the code will typically test many (50+) possible combination of fit parameters!

In [ ]:
# Loop, repeating the data generation and fit:
for iexp in range(Nexp) : 

    # Generating data by filling values into (x,y) and associated uncertainty in y:
    x = np.arange(Npoints)+1
    y = alpha0 + alpha1*x + r.normal(0, sigmay, Npoints) #+ alpha2*x**2
    ey = sigmay*np.ones_like(x)

    # ------------------------------------------------------------------ #
    # Analytical fit:
    # ------------------------------------------------------------------ #
    # Calculate the relevant sums (see note linked to above):
    sum = len(x)
    sumx = np.sum(x)
    sumxx = np.sum(x**2)
    sumy = np.sum(y)
    sumxy = np.sum(x*y)
    
    # Use sums in calculations of the fit parameters:
    Delta = sumxx * sum - sumx**2
    alpha0_calc = (sumy  * sumxx - sumxy * sumx) / Delta
    alpha1_calc = (sumxy * sum   - sumy  * sumx) / Delta
    sigma_alpha0_calc = sigmay * np.sqrt(sumxx / Delta)
    sigma_alpha1_calc = sigmay * np.sqrt(sumx  / Delta)

    # So now you have data points and a fit/theory. How to get the fit quality?
    # The answer is to calculate the Chi2 and Ndof, and from these two values get their
    # probability using the ChiSquare function (e.g. from scipy):
    Chi2_calc = 0.0
    for i in range( Npoints ) : 
        fit_value = alpha0_calc + alpha1_calc * x[i]
        Chi2_calc += ((y[i] - fit_value) / ey[i])**2
    
    Nvar = 2                     # Number of variables (alpha0 and alpha1)
    Ndof_calc = Npoints - Nvar   # Number of degrees of freedom = Number of data points - Number of variables
    
    # From Chi2 and Ndof, one can calculate the probability of obtaining this
    # or something worse (i.e. higher Chi2). This is a good function to have!
    # We will discuss in class, why/how this function works, and how it plays a central role in statistics.
    from scipy import stats
    Prob_calc =  stats.chi2.sf(Chi2_calc, Ndof_calc) # The chi2 probability given N degrees of freedom (Ndof)

    
    # ------------------------------------------------------------------ #
    # Numerical fit (i.e. with iMinuit):
    # ------------------------------------------------------------------ #
    # Define a fit function:
    def fit_function(x, alpha0, alpha1):
        return alpha0 + alpha1*x

    # Now we define a ChiSquare to be minimised, using the iminuit cost function:
    chi2_object_function = LeastSquares(x, y, ey, fit_function)
    
    # Here we let Minuit know, what to minimise, how, and with what starting parameters:   
    mfit = Minuit(chi2_object_function, alpha0=3.0, alpha1=0.5)     # iminuits function

    # Perform the actual fit (without printing anything):
    mfit.print_level = 0
    mfit.migrad()

    # Record the fit parameters:
    alpha0_fit = mfit.values['alpha0']
    alpha1_fit = mfit.values['alpha1']
    sigma_alpha0_fit = mfit.errors['alpha0']
    sigma_alpha1_fit = mfit.errors['alpha1']
    
    # In Minuit, you can just ask the fit function for the chi2 (or likelihood) value:
    Chi2_fit = mfit.fval                          # The chi2 value
    Prob_fit = stats.chi2.sf(Chi2_fit, Ndof_calc) # The chi2 probability given N degrees of freedom (Ndof, taken from above!)
    
    # Let us see what the fit gives for the first couple of datasets:
    if (iexp < 10) :
        print(f"  Ana. Fit: a0={alpha0_calc:6.3f}+-{sigma_alpha0_calc:5.3f}  a1={alpha1_calc:5.3f}+-{sigma_alpha1_calc:5.3f}  p={Prob_calc:6.4f}"+
              f"  Num. Fit: a0={alpha0_fit:6.3f}+-{sigma_alpha0_fit:5.3f}  a1={alpha1_fit:5.3f}+-{sigma_alpha1_fit:5.3f}  p={Prob_fit:6.4f}")


In [ ]:
fig, ax = plt.subplots(figsize=(10,6))
ax.errorbar(x, y, ey, fmt='ro', ecolor='k', elinewidth=1, capsize=2, capthick=1)
ax.plot(x, fit_function(x, *mfit.values), '-r')
plt.close()

fit_info = [f'alpha0 = {alpha0_fit:5.3f}' + r'$\pm$' +  f"{sigma_alpha0_fit:5.3f}",
            f'alpha1 = {alpha1_fit:5.3f}' + r'$\pm$' + f"{sigma_alpha1_fit:5.3f}",
            f'Chi2 = {Chi2_fit:5.3f}',
            f'Ndof = {Ndof_calc:d}',
            f'Prob = {Prob_fit:5.3f}',
]
ax.text(0.05, 0.76, "\n".join(fit_info), fontsize=14, transform = ax.transAxes)
fig.tight_layout()
fig

In [ ]:
if (save_plots) :
    fig.savefig('AnalyticalLinearFit.pdf', dpi=600)

---------------------------------------------------------------------------------- 


# Questions:

1) Do the analytical result(s) (Calc) agree with the numerical one(s) (Fit)?

### Advanced questions:

2) Can you measure/determine the difference in speed between the two methods?